# Pré-processamento Mini-Flight-Delay

In [1]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

### Ler dataframes

In [37]:
df_train = pd.read_csv('archive/flight_delays_train.csv')
#df_test = pd.read_csv('archive/flight_delays_test.csv')

In [38]:
df_train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


### Função para split de valores das 3 primeiras colunas

In [39]:
def split_values(str):
    value = str.split('-')[1]
    return int(value)

### Map dos valores com a função split_values para inteiros

In [40]:
df_train['Month'] = df_train['Month'].map(lambda x: split_values(x))
df_train['DayofMonth'] = df_train['DayofMonth'].map(lambda x: split_values(x))
df_train['DayOfWeek'] = df_train['DayOfWeek'].map(lambda x: split_values(x))

#df_test['Month'] = df_test['Month'].map(lambda x: split_values(x))
#df_test['DayofMonth'] = df_test['DayofMonth'].map(lambda x: split_values(x))
#df_test['DayOfWeek'] = df_test['DayOfWeek'].map(lambda x: split_values(x))

In [41]:
df_train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,8,21,7,1934,AA,ATL,DFW,732,N
1,4,20,3,1548,US,PIT,MCO,834,N
2,9,2,5,1422,XE,RDU,CLE,416,N
3,11,25,6,1015,OO,DEN,MEM,872,N
4,10,7,6,1828,WN,MDW,OMA,423,Y


In [7]:
print(pd.DataFrame(df_train.dtypes))

                        0
Month               int64
DayofMonth          int64
DayOfWeek           int64
DepTime             int64
UniqueCarrier      object
Origin             object
Dest               object
Distance            int64
dep_delayed_15min  object


### Mapear colunas UniqueCarrier, Origin, Dest
- Mapeia valores em um dict, substitui no dataframe com índices inteiros

In [42]:
## Treino
# UniqueCarrier
uc_labels = df_train.UniqueCarrier.unique().tolist()
label_dict_uc_train = {}
for index, possible_label in enumerate(uc_labels):
    label_dict_uc_train[possible_label] = index

# Origin
origin_labels = df_train.Origin.unique().tolist()
label_dict_origin_train = {}
for index, possible_label in enumerate(origin_labels):
    label_dict_origin_train[possible_label] = index

# Dest
dest_labels = df_train.Dest.unique().tolist()
label_dict_dest_train = {}
for index, possible_label in enumerate(dest_labels):
    label_dict_dest_train[possible_label] = index

'''    
## Test
# UniqueCarrier
uc_labels = df_test.UniqueCarrier.unique().tolist()
label_dict_uc_test = {}
for index, possible_label in enumerate(uc_labels):
    label_dict_uc_test[possible_label] = index

# Origin
origin_labels = df_test.Origin.unique().tolist()
label_dict_origin_test = {}
for index, possible_label in enumerate(origin_labels):
    label_dict_origin_test[possible_label] = index

# Dest
dest_labels = df_test.Dest.unique().tolist()
label_dict_dest_test = {}
for index, possible_label in enumerate(dest_labels):
    label_dict_dest_test[possible_label] = index
'''

'    \n## Test\n# UniqueCarrier\nuc_labels = df_test.UniqueCarrier.unique().tolist()\nlabel_dict_uc_test = {}\nfor index, possible_label in enumerate(uc_labels):\n    label_dict_uc_test[possible_label] = index\n\n# Origin\norigin_labels = df_test.Origin.unique().tolist()\nlabel_dict_origin_test = {}\nfor index, possible_label in enumerate(origin_labels):\n    label_dict_origin_test[possible_label] = index\n\n# Dest\ndest_labels = df_test.Dest.unique().tolist()\nlabel_dict_dest_test = {}\nfor index, possible_label in enumerate(dest_labels):\n    label_dict_dest_test[possible_label] = index\n'

In [43]:
df_train['UniqueCarrier'] = df_train.UniqueCarrier.replace(label_dict_uc_train)
df_train['Origin'] = df_train.Origin.replace(label_dict_origin_train)
df_train['Dest'] = df_train.Dest.replace(label_dict_dest_train)

'''
df_test['UniqueCarrier'] = df_test.UniqueCarrier.replace(label_dict_uc_test)
df_test['Origin'] = df_test.Origin.replace(label_dict_origin_test)
df_test['Dest'] = df_test.Dest.replace(label_dict_dest_test)
'''

"\ndf_test['UniqueCarrier'] = df_test.UniqueCarrier.replace(label_dict_uc_test)\ndf_test['Origin'] = df_test.Origin.replace(label_dict_origin_test)\ndf_test['Dest'] = df_test.Dest.replace(label_dict_dest_test)\n"

In [44]:
df_train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,8,21,7,1934,0,0,0,732,N
1,4,20,3,1548,1,1,1,834,N
2,9,2,5,1422,2,2,2,416,N
3,11,25,6,1015,3,3,3,872,N
4,10,7,6,1828,4,4,4,423,Y


### Normalização de valores

In [28]:
df_train['dep_delayed_15min'] = df_train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

In [29]:
from sklearn import preprocessing

att = list(df_train.columns.values)

train_values = df_train.values
scaler = preprocessing.MinMaxScaler()
values_scaled = scaler.fit_transform(train_values)

train_scaled_df = pd.DataFrame(values_scaled,columns=att)

train_scaled_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,0.636364,0.666667,1.000000,0.763127,0.000000,0.000000,0.000000,0.142336,0.0
1,0.272727,0.633333,0.333333,0.610738,0.047619,0.003472,0.003472,0.163017,0.0
2,0.727273,0.033333,0.666667,0.560995,0.095238,0.006944,0.006944,0.078264,0.0
3,0.909091,0.800000,0.833333,0.400316,0.142857,0.010417,0.010417,0.170722,0.0
4,0.818182,0.200000,0.833333,0.721279,0.190476,0.013889,0.013889,0.079684,1.0


### Select de instâncias de treino e teste

In [45]:
X = df_train.iloc[:,:-1].values
y = df_train['dep_delayed_15min'].values

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=14)

In [58]:
df_train.to_csv('df.csv', index=False)

### Testes com classificadores
- Testes com modelos MLP

In [54]:
mlp = MLPClassifier(hidden_layer_sizes=(50,),learning_rate='adaptive',
                    learning_rate_init=0.01, max_iter=100,
                    random_state=14)

In [55]:
mlp.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(50,), learning_rate='adaptive',
              learning_rate_init=0.01, max_iter=100, random_state=14)

In [19]:
from sklearn.metrics import classification_report

In [52]:
y_pred = mlp.predict(X_test)

In [25]:
print(classification_report(y_test, y_pred, target_names=['N', 'Y']))

              precision    recall  f1-score   support

           N       0.88      0.69      0.77     20195
           Y       0.31      0.59      0.41      4805

    accuracy                           0.67     25000
   macro avg       0.59      0.64      0.59     25000
weighted avg       0.77      0.67      0.70     25000



In [36]:
print('Normalizado')
print(classification_report(y_test, y_pred, target_names=['0', '1']))

Normalizado
              precision    recall  f1-score   support

           0       0.88      0.69      0.77     20195
           1       0.31      0.59      0.41      4805

    accuracy                           0.67     25000
   macro avg       0.59      0.64      0.59     25000
weighted avg       0.77      0.67      0.70     25000



In [53]:
print('5 camadas, learning rate adaptativo')
print(classification_report(y_test, y_pred, target_names=['0', '1']))

5 camadas, learning rate adaptativo
              precision    recall  f1-score   support

           0       0.81      1.00      0.89     20195
           1       0.77      0.00      0.01      4805

    accuracy                           0.81     25000
   macro avg       0.79      0.50      0.45     25000
weighted avg       0.80      0.81      0.72     25000



In [56]:
print('50 camadas, learning rate adaptativo')
print(classification_report(y_test, y_pred, target_names=['0', '1']))

50 camadas, learning rate adaptativo
              precision    recall  f1-score   support

           0       0.81      1.00      0.89     20195
           1       0.77      0.00      0.01      4805

    accuracy                           0.81     25000
   macro avg       0.79      0.50      0.45     25000
weighted avg       0.80      0.81      0.72     25000

